In [1]:
import scaper
import jams
import os
import numpy as np
import gzip
import glob
import pandas as pd
from tqdm import tqdm
np.random.seed(123)

In [2]:
metadatafolder = '/scratch/js7561/datasets/scaper_waspaa2017/urban-sed/metadata/'
test_jams_files = sorted(glob.glob(os.path.join(metadatafolder, 'test', '*.jams')))
len(test_jams_files)

2000

In [4]:
for jf in tqdm(test_jams_files):
    
    # Load jam
    jam = jams.load(jf)
    
    # For all SNR groups
    snr_range = 6
    for min_snr in [6, 12, 18, 24]:
        
        # Change SNR of all events
        for i in range(len(jam.annotations[0].data)):
            jam.annotations[0].data.loc[i, 'value']['snr'] = min_snr + np.random.rand() * snr_range
            
        # Save new jam
        outjam = os.path.join(metadatafolder, 'snr', os.path.basename(jf).replace('.jams', '_snr{:d}.jams'.format(min_snr)))
        jam.save(outjam)
        
        # Create event list for txt file
        df = pd.DataFrame(columns=['onset', 'offset', 'label'])

        for idx, row in jam.annotations[0].data.iterrows():
            if row.value['role'] == 'foreground':
                newrow = ([row.time.total_seconds(),
                           row.time.total_seconds() +
                           row.duration.total_seconds(),
                           row.value['label']])
                df.loc[len(df)] = newrow

        # sort events by onset time and save
        outtxt = outjam.replace('.jams', '.txt')
        df = df.sort_values('onset')
        df.reset_index(inplace=True, drop=True)
        df.to_csv(outtxt, index=False, header=False, sep=' ')

100%|██████████| 2000/2000 [06:56<00:00,  5.54it/s]
